<a href="https://colab.research.google.com/github/WSEHUchidaTsubasa/project1/blob/main/Reserch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf

ticker = yf.Ticker("AAPL")
price = ticker.history(period="5y")
price.dropna()

In [ ]:
import pandas as pd
import numpy as np
df=price
trainsize=int(len(df)*0.7)
validsize=int(len(df)*0.15)
train_df=df.iloc[:trainsize]
valid_df=df.iloc[trainsize:trainsize+validsize]
test_df=df.iloc[trainsize+validsize:]


df["target"]=np.where(df["Close"].shift(-1)>df["Close"],1,0)

train_X=df.drop("target",axis=1)
train_y=df["target"]

In [ ]:
def my_rolling_window(n_samples,trainsize,stepsize):
  start_train=0
  end_train=trainsize
  start_valid=end_train
  end_valid=start_valid+stepsize
  while end_valid<n_samples:
    yield(slice(start_train,end_train),slice(start_valid, end_valid))
    start_train+=stepsize
    end_train+=stepsize
    start_valid+=stepsize
    end_valid+=stepsize


In [ ]:
#とりあえず、optunaつかってreserch段階を済まします。
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
!pip install optuna
import optuna
from optuna.samplers import TPESampler

def objective(trial):
  params = {
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "objective": "binary",
        "metric": "binary_logloss",
        "random_state": 42,
        "n_jobs": -1
    }

  scores=[]#np.full(len(train_X),np.nan)

  for tr_idx,val_idx in my_rolling_window(n_samples=len(train_X),trainsize=30,stepsize=30):
    X_tr,X_val=train_X.iloc[tr_idx],train_X.iloc[val_idx]
    y_tr,y_val=train_y.iloc[tr_idx],train_y.iloc[val_idx]
    lgbm=LGBMClassifier(**params)
    lgbm.fit(X_tr,y_tr)

    pred=lgbm.predict_proba(X_val)[:,1]

    score=log_loss(y_val,pred)
    scores.append(score)

    return np.mean(scores)

study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(
    objective,
    n_trials=100,
    show_progress_bar=True
)


In [ ]:
print("Best score:", study.best_value)
print("Best params:", study.best_params)


In [ ]:
best_params=study.best_params

import yaml
config={
    "model":{
        "name":"lgbm",
        "params":best_params
    }
}

with open("config.yaml","w") as f:
  yaml.safe_dump(config,f,sort_keys=False)